Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
!pip install category_encoders

In [2]:
# Import

%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

from sklearn.pipeline import make_pipeline
import category_encoders as ce
from scipy.stats import randint, uniform

In [3]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [4]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [5]:
# Train/Validate/Test split the data
train, val = train_test_split(
    train, train_size=0.80, test_size=0.20,
    stratify=train['status_group'], random_state=4
)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [6]:
sample_submission.shape

(20515, 2)

In [7]:
from math import radians, cos, sin, asin, sqrt

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

In [8]:
# Cluster the locations
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, n_jobs=-1)
train['cluster'] = kmeans.fit_predict(train[['longitude', 'latitude']])
val['cluster'] = kmeans.predict(val[['longitude', 'latitude']])
test['cluster'] = kmeans.predict(test[['longitude', 'latitude']])

In [9]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()

    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                        'population', 'gps_height']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (all the same), id (can't glean useful info), num_private(mostly 0s)
    unused = ['recorded_by']
    X = X.drop(columns=unused)

    # Changing region and district code into categories rather than ints
    X['region_code'] = X['region_code'].astype('str')
    X['district_code'] = X['district_code'].astype('str')

    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
   
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    #binning construction_years
    cut_bins = [0, 1, 1960, 1970, 1980, 1990, 2000, 2010, 2013]
    X['construction_bin'] = pd.cut(x=X['construction_year'], bins= cut_bins, include_lowest=True, duplicates='raise').values.add_categories('missing')
    X['construction_bin']=X['construction_bin'].fillna('missing')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years_till_check'] = X['year_recorded'] - X['construction_year']

    # Engineer feature: Age of waterpoint
    X['age'] = 2013 - X['construction_year']

    # Engineer feature: Water quality and quantity
    X['good_enough'] = ((X['quality_group'] == 'good') & (X['quantity'] == 'enough')).astype(int)
    
    # Engineer feature: Using lat and long features
    # Trying two ways to use the lat and long - haversine and 
    # https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature
    # see which one works better for model
    X['x'] = np.cos(X['latitude']) * np.cos(X['longitude'])
    X['y'] = np.cos(X['latitude']) * np.sin(X['longitude'])
    X['z'] = np.sin(X['latitude'])

    # Engineer feature: Haversine point
    X['harvesine_distance'] = [single_pt_haversine(latitude, longitude) for latitude, longitude in zip(X['latitude'], X['longitude'])]

    # Eeturn the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [10]:
def seasonality(X):
  X = X.copy()

  hot = (X['month_recorded'] == 11)
  hot = (X['month_recorded'] == 12)
  hot = (X['month_recorded'] == 1)
  hot = (X['month_recorded'] == 2)
  rain = (X['month_recorded'] == 3)
  rain = (X['month_recorded'] == 4)
  rain = (X['month_recorded'] == 5)
  cool = (X['month_recorded'] == 6)
  cool = (X['month_recorded'] == 7)
  cool = (X['month_recorded'] == 8)
  cool = (X['month_recorded'] == 9)
  cool = (X['month_recorded'] == 10)

  X['hot'] = hot
  X['rain'] = rain
  X['cool'] = cool

  return X

train = seasonality(train)
val = seasonality(val)
test = seasonality(test)

In [11]:
train.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,years_till_check,age,good_enough,x,y,z,harvesine_distance,hot,rain,cool
57685,51201,5000.0,Wsdp,112.0,Wachina,38.865788,-6.829784,Mshindo Shabani Bom,102,Wami / Ruvu,...,1.0,3.0,0,0.335918,0.785483,-0.519785,4377.473537,True,False,False
19536,71066,500.0,Unicef,1525.0,DWE,34.664787,-8.984294,Shuleni,0,Rufiji,...,27.0,29.0,1,0.899346,0.096843,-0.426377,3966.172575,False,False,True
17706,20440,0.0,Government Of Tanzania,243.0,DWE,38.560631,-9.780829,Barabara Ya Rwangwa,0,Ruvuma / Southern Coast,...,13.0,13.0,0,-0.610437,-0.711239,0.348576,4402.620027,False,False,False
38260,6595,0.0,Netherlands,NaN,DWE,34.062483,-3.232824,Upendo,0,Internal,...,NaN,NaN,1,0.876292,-0.473088,0.091105,3802.545152,False,False,True
43503,62370,0.0,Rwssp,NaN,RWSSP,33.204758,-3.866347,Kwa Kangayu,0,Internal,...,NaN,NaN,0,0.161945,-0.730937,0.662951,3714.296528,False,False,True


In [12]:
# Select features

# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.drop('id').tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality.index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)
print(len(features))

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'population', 'construction_year', 'cluster', 'year_recorded', 'month_recorded', 'day_recorded', 'years_till_check', 'age', 'good_enough', 'x', 'y', 'z', 'harvesine_distance', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'longitude_MISSING', 'latitude_MISSING', 'construction_year_MISSING', 'population_MISSING', 'gps_height_MISSING', 'construction_bin', 'hot', 'rain', 'cool']
55


In [13]:
# Arrange data into X features matrix and y target vector

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [15]:
#Random Forest Classifier 
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names = True, cols = ['basin']),
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state = 42),
)


In [16]:
#Hyperparameter optimization with RandomSearchCV

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1),
}

In [17]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions = param_distributions,
    n_iter = 10,
    cv = 5,
    scoring = 'accuracy',
    verbose = 10,
    return_train_score = True,
    n_jobs = -1
)

search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 45.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 50.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 54.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 58.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 60.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 68.5min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('onehotencoder',
                                              OneHotEncoder(cols=['basin'],
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            return_df=True,
                                                            use_cat_names=True,
                                                            verbose=0)),
                                             ('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                      

In [18]:
print('Best H yperparameters:', search.best_params_)
print('Accuracy:', search.best_score_)
print('Best Estimator:', search.best_estimator_)

Best H yperparameters: {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.23436362532250588, 'randomforestclassifier__n_estimators': 296, 'simpleimputer__strategy': 'mean'}
Accuracy: 0.8096801346801347
Best Estimator: Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=True, verbose=0)),
                ('ordinalencoder',
                 OrdinalEncoder(cols=['funder', 'installer', 'wpt_name',
                                      'subvillage', 'region', 'region_code',
                                      'district_code', 'lga', 'ward',
                                      'public_meeting', 'scheme_ma...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_dep

In [19]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,7,3,4,9,6,1,2,0,5,8
mean_fit_time,84.5015,118.472,159.955,274.055,40.855,1491.17,437.817,333.004,121.825,12.0502
std_fit_time,0.646092,1.00063,1.42563,51.6649,0.374502,355.923,153.374,9.64676,2.60448,0.0884381
mean_score_time,1.67265,1.47285,1.97565,1.56173,1.02086,9.27251,1.77164,2.93346,0.95337,0.709395
std_score_time,0.0668498,0.0305448,0.0539382,0.314693,0.0164058,4.45586,0.23012,0.530841,0.0289043,0.0201619
param_randomforestclassifier__max_depth,20,20,15,20,15,None,15,None,None,10
param_randomforestclassifier__max_features,0.234364,0.377717,0.342029,0.709382,0.228311,0.640457,0.771114,0.735366,0.843433,0.112192
param_randomforestclassifier__n_estimators,296,268,454,364,155,494,347,114,114,79
param_simpleimputer__strategy,mean,mean,mean,mean,median,median,median,mean,mean,median
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 10, 'ran..."
split0_test_score,0.81515,0.812099,0.806523,0.809784,0.804734,0.806312,0.803787,0.803156,0.801683,0.762125


In [20]:
#Submission
pipeline = search.best_estimator_
y_pred = pipeline.predict(X_test)

In [21]:
assert all(X_train.columns == X_test.columns)

In [24]:
submission = sample_submission.copy()
# submission['status_group'] = y_pred
# submission.to_csv('Obukwelu_DSPT5_Unit_2_submission3.csv', index=False)

In [ ]:
submission.head()

In [23]:
submission.shape

(20515, 2)

#### Engineering new features

Test features independently and see how what it yields

* gps_height
* working with K-Nearest Neighbors
* or K-Clusters
* working with seasons
* working with population


**try this with your dataset**

